## Optical Flow 
**Optical flow is the pattern of apparent motion of image object between two consecutive frames cause by the movement of object or camera**

* Optical flow analysis has few assumptions:
    
    1. The pixel intensities of an object do not change between consecutive frames.
    
    2. Neighbors pixels have similar motion.
    
    3. The optical flow methods in OpenCV will first take in a given set of points and a frame.
    
    4. Then it will attempt to find those points in the next frame.
    
    5. It is upto the user to supply the points to track.

## Lucas-Kanada
Lucas-Kanada is a function used in Object tracking.

Using OpenCV we pass in the previous frame, previous points and the current frame to the Lucas-Kanada Function.

The Lucas-Kanada computes Optical flow for a sparse feature set.



**But what if we want to track all the points in a video?**

## Gunner Farneback's Algorithm

* We can use Gunner Farnback's algorithm(also built-in to OpenCV) to calculate dense optical flow.

* This dense optical flow will calculate flow for all the points in an image.

* It will color them black if no flow(no movement is detected).

In [1]:
# Import tools
import cv2
import numpy as np

### Lucas-Kanada Algorithm

In [2]:
# Corner tracking parameter dictionary
corner_track_params=dict(maxCorners=10,qualityLevel=0.3,minDistance=7,blockSize=7)

#lucas-kanada parameters dictionary
lk_params=dict(winSize=(200,200),maxLevel=2,criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT,10,0.03))

In [3]:
cap=cv2.VideoCapture(0)

# 1st Frame that will act as the previous frame to the current frame

ret,prev_frame=cap.read()

# Turn this frame to gray scale
prev_gray=cv2.cvtColor(prev_frame,cv2.COLOR_BGR2GRAY)

# Points to track

prevPts=cv2.goodFeaturesToTrack(prev_gray,mask=None,**corner_track_params)

mask=np.zeros_like(prev_frame)

while True:
    # Current frame
    ret, frame=cap.read()
    
    # Convert frame to gray scale 
    
    frame_gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    
    # Calculate the Optical flow  ()
    
    nextPts, status, err= cv2.calcOpticalFlowPyrLK(prev_gray,frame_gray,prevPts,None,**lk_params)
    
    # Status array will give vectors in output vector is set to 1 if any motion is detected between the two frames
    
    good_new=nextPts[status==1]
    good_prev=prevPts[status==1]
    
    for i,(new,prev) in enumerate(zip(good_new,good_prev)):
        x_new,y_new=new.ravel()
        x_prev,y_prev=prev.ravel()
        
        # draw green lines between points in changed frames after apparent motion
        mask=cv2.line(mask,(x_new,y_new),(x_prev,y_prev),(0,255,0),3)
        
        frame=cv2.circle(frame,(x_new,y_new),6,(0,255,0),-1)
        
        
    img=cv2.add(frame,mask)
    
    cv2.imshow('tracking',img)
    
    k=cv2.waitKey(20) & 0xFF
    if k==27:
        break
    
    # updating frames and previous points
    prev_gray=frame_gray.copy()
    prevPts=good_new.reshape(-1,1,2)
    
cv2.destroyAllWindows()

cap.release()

## Dense Optical Flow

**We will use Gunner Farneback's Algo for Dense Optical Flow**

**Parameters**

* prev : First input image in 8-bit single channel format.
    
* next : Second input image of same type and same size as prev.
    
* pyr_scale : parameter specifying the image scale to build pyramids for each image (scale < 1). A classic pyramid is of generally 0.5 scale, every new layer added, it is halved to the previous one.
    
* levels : levels=1 says, there are no extra layers (only the initial image) . It is the number of pyramid layers including the first image.
    
* winsize : It is the average window size, larger the size, the more robust the algorithm is to noise, and provide fast motion detection, though gives blurred motion fields.
    
* iterations : Number of iterations to be performed at each pyramid level.
    
* poly_n : It is typically 5 or 7, it is the size of the pixel neighbourhood which is used to find polynomial expansion between the pixels.
    
* poly_sigma : standard deviation of the gaussian that is for derivatives to be smooth as the basis of the polynomial expansion. It can be 1.1 for poly= 5 and 1.5 for poly= 7.
    
* flow : computed flow image that has similar size as prev and type to be CV_32FC2.
    
* flags : It can be a combination of:
        
    1. OPTFLOW_USE_INITIAL_FLOW uses input flow as initial apporximation.
    
    2. OPTFLOW_FARNEBACK_GAUSSIAN uses gaussian winsize*winsize filter

In [3]:
import numpy as np
import cv2

In [2]:
cap=cv2.VideoCapture(0)

ret,frame1=cap.read()

prev_img=cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)

hsv_mask=np.zeros_like(frame1)

# Grab the saturation and assign highest saturation to the  mask
hsv_mask[:,:,1]=255

while True:
    ret,frame2=cap.read()
    
    next_img=cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)
    
    # use gunner algo to find optical flow
    flow=cv2.calcOpticalFlowFarneback(prev_img,next_img,None,0.5,3,15,3,5,1.2,0)
    
    #After this fuction we have flow object as an output which contains vector flow cartesian information.
    # And we need to convert this output into magnitude and angle
    
    mag,ang=cv2.cartToPolar(flow[:,:,0],flow[:,:,1],angleInDegrees=True)
    
    hsv_mask[:,:,0]=ang/2
    hsv_mask[:,:,2]=cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
    
    bgr=cv2.cvtColor(hsv_mask,cv2.COLOR_HSV2BGR)
    
    cv2.imshow('frame',bgr)
    
    k=cv2.waitKey(20) & 0xFF
    if k==27:
        break
        
    prev_img=next_img
    
cap.release()
cv2.destroyAllWindows()